In [1]:
import numpy as np
from tqdm import tqdm
from solver.qp import qp
import pandas as pd
import torch
import gzip
import pickle
import warnings
from generate_instances import generate_softmarginsvm, generate_markowitz_portfolio_optimization, generate_convex_function_to_data, generate_regressor_selection

### Soft margin Support Vector Machine

In [1]:
def surrogate_gen():
    C = 10 #regularization parameter
    m = 4 #number of Datapoints
    X = np.random.uniform(-1, 1, (m, 2)) #Dataset
    bias = np.full((m, 1), 0.5)
    X[:, 1] += bias.ravel()   # Adding a bias to make data linearly separable
    y = np.array([1.0 if x[0] > x[1] else -1.0 for x in X]).reshape(m, 1) #label vector
    Q, q, G, h, A, b, S = generate_softmarginsvm(y, X, C)    
    return Q, q, G, h, A, b, S

### Markowitz portfolio optimization

In [2]:
def surrogate_gen():
    np.random.seed(1)
    n_obs = 252
    n_assets = 4
    artificial_returns = np.random.randn(n_obs, n_assets) + 0.05 # minimum expected return
    returns = pd.DataFrame(artificial_returns)
    pbar = np.array(returns.mean()).reshape(1, -1)
    r_min = pbar.mean() # minimum return threshold
    Q, q, G, h, A, b, S = generate_markowitz_portfolio_optimization(artificial_returns, r_min, pbar)
    return Q, q, G, h, A, b, S

### Fiting convex function to given data

In [2]:
def surrogate_gen():
    
    # Generate 'u' values (from -2 to 2) for the independent variable
    u_values = np.linspace(-2.0, 2.0, num=50).reshape(-1, 1)
    Q, q, G, h, A, b, S = generate_convex_function_to_data(u_values)
    return Q, q, G, h, A, b, S

### Regressor selection problem

In [4]:
def surrogate_gen():
    
    # Set dimensions
    m = 10  # number of rows for A
    n = 20  # number of columns for A
    nopts = 100 # range of alpha values
    alpha_index = np.random.randint(0, nopts) # Select an alpha value from the range nopts
    Q, q, G, h, A, b, S = generate_regressor_selection(alpha_index, m, n)
    
    return Q, q, G, h, A, b, S

### create

In [5]:
import os

# directory = f'../../../../work/log1/darius.weber/Quadratic_Programming_Datasets/raw'
directory = f'Quadratic_Programming_Datasets/raw'
os.makedirs(directory, exist_ok=True)
warnings.filterwarnings("error")

ips = []
pkg_idx = 0 #identifier index
success_cnt = 0
fail_cnt = 0
max_ipm_steps = 0


max_iter = 100
num = 10

for i in tqdm(range(max_iter)):
    Q, q, G, h, A, b, S = surrogate_gen()
    print(np.array(A).size)
    try:
        res = qp(Q, q, G, h, A, b, callback=lambda res: res)
        max_ipm_steps = max(max_ipm_steps, res['iterations'])
        print("status",res['status'])
    except Exception as e:
        fail_cnt += 1
        warnings.warn(f'Optimization failed with error: {str(e)}')
        continue
    else:
        if res['status'] == 'optimal': #np.isnan(res.fun)?
            ips.append((torch.from_numpy(np.array(Q)).to(torch.float), torch.from_numpy(np.array(q)).to(torch.float), torch.from_numpy(np.array(G)).to(torch.float), torch.from_numpy(np.array(h)).to(torch.float), torch.from_numpy(np.array(A)).to(torch.float), torch.from_numpy(np.array(b)).to(torch.float), torch.from_numpy(np.array(S)).to(torch.float), res))
            print(success_cnt)
            success_cnt += 1
            
    if len(ips) >= 1000 or success_cnt == num:
        print("success")
        ips = [ip + (max_ipm_steps,) for ip in ips]
        with gzip.open(f'{directory}/instance_{pkg_idx}.pkl.gz', "wb") as file:
            pickle.dump(ips, file)
            print(f'Package {pkg_idx} saved')
            pkg_idx += 1

        ips = []
    if success_cnt >= num:
        break
warnings.resetwarnings()

  8%|▊         | 8/100 [00:00<00:02, 42.35it/s]

0
     pcost       dcost       gap    pres   dres
 0: -2.4960e+00 -5.3688e+00  6e+01  5e+00  6e-01
 1: -2.2502e+00 -4.9625e+00  8e+00  5e-01  5e-02
 2: -1.9750e+00 -3.0145e+00  1e+00  2e-02  2e-03
 3: -2.2953e+00 -2.5385e+00  2e-01  9e-17  1e-15
 4: -2.4141e+00 -2.4582e+00  4e-02  3e-16  6e-16
 5: -2.4369e+00 -2.4426e+00  6e-03  2e-16  3e-16
 6: -2.4408e+00 -2.4409e+00  1e-04  1e-16  5e-16
 7: -2.4409e+00 -2.4409e+00  1e-06  2e-16  3e-15
Optimal solution found.
status optimal
0
0
     pcost       dcost       gap    pres   dres
 0: -2.2465e+00 -7.0390e+00  6e+01  3e+00  7e-01
 1: -1.9771e+00 -6.5663e+00  8e+00  2e-01  4e-02
 2: -1.9771e+00 -3.2771e+00  2e+00  3e-02  6e-03
 3: -2.2679e+00 -2.6511e+00  4e-01  9e-17  1e-15
 4: -2.3334e+00 -2.3935e+00  6e-02  2e-16  2e-16
 5: -2.3464e+00 -2.3576e+00  1e-02  2e-16  1e-16
 6: -2.3480e+00 -2.3493e+00  1e-03  3e-16  1e-16
 7: -2.3481e+00 -2.3481e+00  4e-05  1e-16  2e-16
 8: -2.3481e+00 -2.3481e+00  4e-07  3e-16  2e-16
Optimal solution found.
st

  9%|▉         | 9/100 [00:00<00:04, 19.44it/s]

Package 0 saved


In [ ]:
# Open and load the .pkl.gz file
with gzip.open(f'Quadratic_Programming_Datasets/raw/instance_{pkg_idx-1}.pkl.gz', 'rb') as f:
    data = pickle.load(f)

# Now `data` contains the contents of the pickle file
print(data)